In [1]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pymongo
import time
import datetime

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.housing_db
collection = db.housing
collection1 = db.redfin

In [7]:
# URL of page to be scraped
url = 'https://www.redfin.com/news/data-center/'

# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')
soup

<html>
<head><title>403 Forbidden</title></head>
<body>
<center><h1>403 Forbidden</h1></center>
<hr/><center>nginx</center>
</body>
</html>

In [5]:
# Locate the desired  'a' tags
one_a_tag= soup.findAll('a',target="_blank")[1]
one_a_tag

IndexError: list index out of range

In [ ]:
# Extract the actual link
download_url = one_a_tag['href']
download_url

In [ ]:
# Load into a pandas dataframe
data = pd.read_csv(download_url, sep ='\t')
data

In [ ]:
list(data.columns)

In [ ]:
data.reset_index(inplace = True)
data_dict1 = data.to_dict("records")

In [ ]:
 # Insert dictionary into MongoDB as a document
    collection1.insert_many(data_dict1)

In [ ]:
homeprices_df = data[['period_end','region_type','region_name','total_homes_sold','total_homes_sold_yoy','median_sale_price','median_sale_price_yoy','inventory','percent_homes_sold_above_list','median_sale_ppsf']]
homeprices_df.head()

In [ ]:
homeprices_df = homeprices_df.rename(columns={"period_end":"Month of Period End","region_type":"Region Type","region_name":"Region","total_homes_sold": "Homes Sold","total_homes_sold_yoy":"Homes Sold YOY","median_sale_price":"Median Sale Price",
                                                  "median_sale_price_yoy":"Median Sale Price YOY","inventory":"Inventory",
                                                   "percent_homes_sold_above_list": "Homes Sold Above List price","median_sale_ppsf":"Price Per Square Foot"})
homeprices_df.head()

In [ ]:
homeprices_df.reset_index(inplace = True)
data_dict = homeprices_df.to_dict("records")

In [ ]:
 # Insert dictionary into MongoDB as a document
    collection.insert_many(data_dict)